# DATA PREPARATION!

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import box,Polygon



      fid                                         denom  \
0     1.0  Scuola dellinfanzia (st.) di Musile di Piave   
1     2.0      Scuola dellinfanzia (par.) Decor Carmeli   
2     6.0             Scuola primaria (st.) Tito Acerbo   
3     4.0            Scuola primaria (st.) E. De Amicis   
4     7.0         Scuola secondaria di I° (st.) E. Toti   
...   ...                                           ...   
2158  NaN                                          None   
2159  NaN                                          None   
2160  NaN                                          None   
2161  NaN                                          None   
2162  NaN                                          None   

                                        indirizzo               tipologia  \
0                  Musile di Piave, Via 29 Aprile          Scuola materna   
1     Musile di Piave (Croce), Piazza T. Acerbo 1          Scuola materna   
2           Musile di Piave (Croce), Via Bosco 13       Scuo

# 数据预处理

In [ ]:
# 设置格网大小（单位和你的坐标系一致，比如米）
grid_size = 1000  # 每个正方形边长1000米（1公里）

# 获取工厂数据的包络范围 (bounding box)
minx, miny, maxx, maxy = gdf_boundary.total_bounds

# 创建格网
grid_cells = []
x_left = minx
while x_left < maxx:
    y_bottom = miny
    while y_bottom < maxy:
        cell = box(x_left, y_bottom, x_left + grid_size, y_bottom + grid_size)
        grid_cells.append(cell)
        y_bottom += grid_size
    x_left += grid_size

# 转成GeoDataFrame
grid = gpd.GeoDataFrame({'geometry': grid_cells}, crs=gdf_processing.crs)

## 空间连接

In [ ]:
# 对于点的处理
# 每个格子ID对应工厂数
join = gpd.sjoin(gdf_processing, grid, how='left', predicate='within')
grid['factory_count'] = join.groupby('index_right').size()
grid['factory_count'] = grid['factory_count'].fillna(0)
# 每个格子ID对应公共服务
join = gpd.sjoin(gdf_services, grid, how='left', predicate='within')
grid['public_services'] = join.groupby('index_right').size()
grid['public_services'] = grid['public_services'].fillna(0)
# 每个格子ID对应废物处理
join = gpd.sjoin(gdf_waste, grid, how='left', predicate='within')
grid['waste_treatment'] = join.groupby('index_right').size()
grid['waste_treatment'] = grid['waste_treatment'].fillna(0)



# 对于线的处理
join = gpd.sjoin(gdf_roads, grid, predicate='intersects')
join['length'] = join.geometry.length
grid['road_length'] = join.groupby('index_right')['length'].sum()
grid['road_length'] = grid['road_length'].fillna(0)



# 对于面的处理
# 每个格子ID对应industrial_area的面积
join = gpd.sjoin(gdf_industrial, grid, predicate='intersects')
join['intersect_area'] = join.geometry.intersection(join['geometry']).area
industrial_area_by_grid = join.groupby('index_right')['intersect_area'].sum()
grid['industrial_area'] = grid.index.map(industrial_area_by_grid).fillna(0)
# 每个格子ID对应industrial_area的面积
join = gpd.sjoin(gdf_agriculture, grid, predicate='intersects')
print(join)
join['intersect_area'] = join.geometry.intersection(join['geometry']).area
agriculture_area_by_grid = join.groupby('index_right')['intersect_area'].sum()
grid['agriculture_area'] = grid.index.map(agriculture_area_by_grid).fillna(0)
# 每个格子ID对应water的面积
join = gpd.sjoin(gdf_water, grid, predicate='intersects')
join['intersect_area'] = join.geometry.intersection(join['geometry']).area
water_area_by_grid = join.groupby('index_right')['intersect_area'].sum()
grid['water_area'] = grid.index.map(water_area_by_grid).fillna(0)


print(grid)

#
# fig, ax = plt.subplots(figsize=(12, 12))
# grid.plot(column='factory_count', cmap='Blues', edgecolor='grey', linewidth=0.5, ax=ax, legend=True)
# # gdf_buildings.plot(ax=ax, color='red', markersize=5)
# plt.title("Factory Counts per Grid Cell", fontsize=15)
# plt.show()

In [ ]:
variables = ['factory_count', 'public_services','waste_treatment','road_length', 'industrial_area','agriculture_area','water_area']

fig, axs = plt.subplots(2, 4, figsize=(18, 6))
axs = axs.flatten()
for i, var in enumerate(variables):
    grid.plot(column=var, ax=axs[i], cmap='OrRd', legend=True, edgecolor='grey', linewidth=0.2)
    axs[i].set_title(f'{var} per Grid', fontsize=12)
    axs[i].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
import seaborn as sns
import pandas as pd

# 只提取需要分析的字段
df_analysis = grid[['factory_count', 'public_services','waste_treatment','road_length', 'industrial_area','agriculture_area','water_area']]
print(df_analysis)
# 计算相关性矩阵
corr_matrix = df_analysis.corr()
corr_matrix.to_csv("correlation_matrix.csv")
# 热力图可视化
plt.figure(figsize=(6, 5))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Matrix")
plt.show()

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_analysis)
kmeans = KMeans(n_clusters=6, random_state=0)
labels = kmeans.fit_predict(X_scaled)
grid['cluster'] = labels

fig, ax = plt.subplots(figsize=(10, 10))
grid.plot(column='cluster', cmap='Set1', legend=True, ax=ax, edgecolor='grey')
ax.set_title("K-Means Clustering by Urban Structure")
ax.axis('off')
plt.tight_layout()
plt.show()

cluster_means = pd.DataFrame(X_scaled, columns=df_analysis.columns)
cluster_means['cluster'] = labels
result = cluster_means.groupby('cluster').mean()
print(result)

# 建筑体块

In [ ]:
from scipy.spatial import cKDTree
# gdf_processing + gdf_buildings
joined = gpd.sjoin( gdf_processing , gdf_buildings, how='left', predicate='within')
print(joined['index_right'].notnull().sum(), "matched to buildings")
print(joined['index_right'].isnull().sum(), "not matched")

# 获取建筑的质心（用于快速索引）
building_centroids = gdf_buildings.geometry.centroid
building_coords = np.array([(p.x, p.y) for p in building_centroids])
tree = cKDTree(building_coords)
# 对每个加工厂找到最近的建筑索引
factory_coords = np.array([(p.x, p.y) for p in gdf_processing.geometry])
dists, idxs = tree.query(factory_coords, k=1)

# 提取匹配的建筑属性
nearest_geometries = gdf_buildings.geometry.iloc[idxs].reset_index(drop=True)

# 合并建筑属性到加工厂 GeoDataFrame
gdf_processing['building_geom'] = nearest_geometries
gdf_processing = gdf_processing[['DENOMINAZI', 'Type', 'geometry', 'building_geom']]
gdf_processing_building = gdf_processing[['DENOMINAZI', 'Type', 'building_geom']]

# # 绘图
# fig, ax = plt.subplots(figsize=(30,30))
#
# # 建筑面（Polygon）
# gdf_buildings.plot(ax=ax, color='grey')
#
# # 加工厂点（Point）
# gdf_processing.plot(ax=ax, color='red', markersize=20, alpha=0.8)
#
# # 额外可视设置
# ax.set_title("Food Processing Sites and Buildings", fontsize=15)
# ax.axis('off')
#
# plt.tight_layout()
# plt.show()

In [ ]:
gdf_processing.to_csv("origin_data/processing.csv", index=False)
gdf_processing_building .to_file("origin_data/processing_building.shp")